# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:

import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

1265


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:

try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS clement
        WITH REPLICATION = { 'class':'SimpleStrategy', 'replication_factor':1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:

try:
    session.set_keyspace('clement')
except Exception as e:
    print(e)

In [8]:
import pandas
df = pandas.read_csv("event_datafile_new.csv")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## Since the data is uniquely identified by the combination of sessionId and itemInSession, these two columns form the Composite Primary Key of the table.
q1create = 'CREATE TABLE IF NOT EXISTS song_sessions'
q1create = q1create + '(session_id int, item_in_session int, artist_name text, song_title text, song_duration decimal, PRIMARY KEY(session_id, item_in_session) \
                         )'

try:
    session.execute(q1create)
except Exception as e:
    print(e)

                    

In [10]:
df[(df["sessionId"] == 338) & (df["itemInSession"] == 4)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId


In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO song_sessions(session_id, item_in_session, artist_name, song_title, song_duration)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, ( int(line[8]), int(line[3]), line[0], line[9], float(line[5]) ))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## query to select artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
q1select = "select artist_name, song_title, song_duration from song_sessions where session_id = 338 and item_in_session = 4"

try:
    rows = session.execute(q1select)
except Exceptions as e:
    print(e)

for row in rows:
    print(row.artist_name, row.song_title, row.song_duration )

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
# We use the sessionId as the Primary Key and userId and itemInSession as the clustering columns
# The query will first use userId and then sessionId in its where clause.  
q2create = 'CREATE TABLE IF NOT EXISTS song_users'
q2create = q2create + '(user_id int, session_id int, artist_name text, song_title text, item_in_session int,\
                        user_firstname text, user_lastname text\
                      , PRIMARY KEY ((user_id, session_id), item_in_session)) )'

try:
    session.execute(q2create)
except Exception as e:
    print(e)
                
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_users(user_id, session_id, artist_name, song_title, item_in_session, user_firstname, user_lastname)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, ( int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4] ))

                    

<Error from server: code=2000 [Syntax error in CQL query] message="line 1:259 mismatched input ')' expecting EOF (..., sessionId), itemInSession)) [)])">


InvalidRequest: Error from server: code=2200 [Invalid query] message="unconfigured table song_users"

In [14]:
## query to select name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
q2select = "select artist_name, song_title, item_in_session, user_firstname, user_lastname \
            from song_users where user_id = 10 and session_id = 182"

try:
    rows = session.execute(q2select)
except Exceptions as e:
    print(e)

for row in rows:
    print(row.artist_name, row.song_title, row.item_in_session, row.user_firstname, row.user_lastname )

Down To The Bone Keep On Keepin' On 0 Sylvie Cruz
Three Drives Greece 2000 1 Sylvie Cruz
Sebastien Tellier Kilometer 2 Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 3 Sylvie Cruz


In [15]:
df[(df["userId"] == 10) & (df["sessionId"] == 182)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
1249,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
1250,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
1251,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
1252,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


In [16]:
# Since a two users can share the same name, we need to use their userId as the unique identifier along with song.  Even though the userId is not required in the actual query.
# We therefore have song as the primary key and userId as the clustering column
q3create = 'CREATE TABLE IF NOT EXISTS song_title'
q3create = q3create + '(song_title text, user_id int, user_firstname text, user_lastname text \
    , PRIMARY KEY(song_title, user_id) )'

try:
    session.execute(q3create)
except Exception as e:
    print(e)
                
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_title(song_title,user_id, user_firstname, user_lastname)"
        query = query + "VALUES(%s, %s, %s, %s)"
        session.execute(query, ( line[9],int(line[10]), line[1], line[4] ))

                    

In [17]:
## query to select every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
q3select = """SELECT user_firstname,
              user_lastname
              FROM song_title
              WHERE song_title = 'All Hands Against His Own';"""

try:
    rows = session.execute(q3select)
except Exceptions as e:
    print(e)

for row in rows:
    print(row.song_title, row.user_firstname, row.user_lastname  )

In [18]:
df[df["song"] == 'All Hands Against His Own']

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId


### Drop the tables before closing out the sessions

In [80]:
query1 = "drop table song_sessions"
query2 = "drop table song_users"
query3 = "drop table song_title"

try:
    rows1 = session.execute(query1)
    rows2 = session.execute(query2)
    rows3 = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [81]:
session.shutdown()
cluster.shutdown()